# Vaihe 1

### Kiinnostavat tutkimuskysymykset
1. Mitkä genret ovat olleet suosittuja milloin?
2. Mikä näyttelijöiden yhdistelmä tuottaa suosittuja elokuvia?
3. Korrelaatio arvioiden ja tuoton välillä. Voiko elokuva olla huonosti arvioitu mutta silti tuottoisa tai toisin päin?
4. Jos tekisimme elokuvan, millainen sen kannattaa olla, että se on tuottoisa ja suosittu?

# Vaihe 2

### Aineiston kuvaus
Aineisto löydettiin Kagglesta ja se sisältää tietoa elokuvista IMDB palvelussa.

Linkki aineistoon: [LINKKI](https://www.kaggle.com/datasets/ashpalsingh1525/imdb-movies-dataset)


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# set display options
pd.set_option('display.max_colwidth', None)

dataset = pd.read_csv('Datasets/imdb_movies.csv')

# TODO: Fixaa tää silleen et selitetään otsikoiden tarkoitus
# Print dataframe columns
print("Aineiston otsikot:")
print(dataset.columns)
print("Otsikoiden tyypit:")
print(dataset.dtypes)
print("Aineiston muoto:")
print(dataset.shape)
amount_of_rows_pre = dataset.shape[0]


Aineiston otsikot:
Index(['names', 'date_x', 'score', 'genre', 'overview', 'crew', 'orig_title',
       'status', 'orig_lang', 'budget_x', 'revenue', 'country'],
      dtype='object')
Otsikoiden tyypit:
names          object
date_x         object
score         float64
genre          object
overview       object
crew           object
orig_title     object
status         object
orig_lang      object
budget_x      float64
revenue       float64
country        object
dtype: object
Aineiston muoto:
(10178, 12)


#### Aineiston otsikot:
- names:        Elokuvan nimi englanniksi
- date_x:       Elokuvan julkaisupäivämäärä
- score:        IMDB-käyttäjien antaman pistemäärän keskiarvo 
- genre:        Elokuvan genre
- overview:     Elokuvan kuvaus englanniksi
- crew:         Elokuvan näyttelijät ja heidän esittämät hahmot. esim. (näyttelijä, hahmo, näyttelijä, hahmo)
- orig_title:   Elokuvan nimi alkuperäisellä kielellä
- status:       Elokuvan tilanne. (Kaikki havainnot ovat julkaistu)
- orig_lang:    Elokuvan alkuperäiskieli
- budget_x:     Elokuvan budjetti (USD)
- revenue:      Elokuvan bruttotuotto (USD)
- country:      Elokuvan julkaisumaa

Aineistossa on siis 12 otsikkoa ja 10178 havaintoa.


Osa aineiston datasta on epäilyttävää. Esimerkiksi 144 elokuvista on saanut 0.0 arvostelun, ja niiden kaikkien tuotto on tasan $175,269,998.80

Aineiston julkaisija on ilmeisesti käyttänyt jotain knn keinoa aineiston täyttämiseen. Tämä täytyy ottaa huomioon myöhemmin.



# Vaihe 3
## Aineiston esikäsittely

#### Huonojen havaintojen poisto

In [2]:
# Tässä poistetaan ne havainnot joissa on tyhjiä arvoja tai arvostelu on 0.0
dataset = dataset.dropna()
dataset = dataset[dataset['score'] != 0.0]
print(dataset.shape)
amount_of_rows_post = dataset.shape[0]
print("Poistettu " + str(amount_of_rows_pre - amount_of_rows_post) + " riviä, jossa oli tyhjiä arvoja tai arvostelu oli 0.0")

(9889, 12)
Poistettu 289 riviä, jossa oli tyhjiä arvoja tai arvostelu oli 0.0


#### Avataan näyttelijät
Emme ole kiinnostuneita näyttelijöiden hahmoista, joten poistetaan ne aineistosta.

In [3]:
print("1. elokuvan näyttelijät:")
print(dataset['crew'].head(1))

1. elokuvan näyttelijät:
0    Michael B. Jordan, Adonis Creed, Tessa Thompson, Bianca Taylor, Jonathan Majors, Damien Anderson, Wood Harris, Tony 'Little Duke' Evers, Phylicia Rashād, Mary Anne Creed, Mila Davis-Kent, Amara Creed, Florian Munteanu, Viktor Drago, José Benavidez Jr., Felix Chavez, Selenis Leyva, Laura Chavez
Name: crew, dtype: object


In [4]:
# Tässä avataan näyttelijät. Näyttelijän hahmo on listattu näyttelijän jälkeen. (esim. Tom Cruise, Ethan Hunt, ...)
dataset['crew'] = dataset['crew'].str.split(', ')
# Poista joka toinen alkio
dataset['crew'] = dataset['crew'].apply(lambda x: x[::2])
print("1. elokuvan näyttelijät:")
print(dataset['crew'].head(1))

1. elokuvan näyttelijät:
0    [Michael B. Jordan, Tessa Thompson, Jonathan Majors, Wood Harris, Phylicia Rashād, Mila Davis-Kent, Florian Munteanu, José Benavidez Jr., Selenis Leyva]
Name: crew, dtype: object


Crew -sarake sisältää siis nyt vain näyttelijät ilman hahmoja

# Vaihe 4
### Aineiston analysointi

In [37]:
#  JOKAISESSA ANALYSOINTIKOHDASSA TEHDÄÄN AINEISTOSTA KOPIO, ETTEI MYÖHEMMÄT KOHDAT MUUTU

#### Mitkä genret ovat suosittuja milloin

In [ ]:
dataset_genre = dataset.copy()

# Extract year from the release date
dataset_genre['date_x'] = pd.to_datetime(dataset_genre['date_x'])
dataset_genre['year'] = dataset_genre['date_x'].dt.year

# Split genres
dataset_genre['genre'] = dataset_genre['genre'].str.split(', ')
dataset_genre = dataset_genre.explode('genre')

# Group by genre and year
grouped = dataset_genre.groupby(['genre', dataset_genre['year'].dt.year]).size().unstack()

# Plot the results
grouped.plot(kind='bar', stacked=True, figsize=(15, 10))
plt.title('Poplarity of movie genres over the years')
plt.ylabel('Amount of movies')
plt.xlabel('Year')
plt.legend(title='Genre', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

C:\Users\aarni\AppData\Local\Temp\ipykernel_11092\1485007041.py:19: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


#### Mikä näyttelijöiden yhdistelmä tuottaa suosittuja elokuvia?

#### Korrelaatio arvioiden ja tuoton välillä. Voiko elokuva olla huonosti arvioitu mutta silti tuottoisa tai toisin päin?

#### Jos tekisimme elokuvan, millainen sen kannattaa olla, että se on tuottoisa ja suosittu?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Copy the dataset
dataset_profitable = dataset.copy()

# Drop rows with missing values
dataset_profitable = dataset_profitable.dropna()
dataset_profitable = dataset_profitable[dataset_profitable['score'] != 0.0]

# Extract year from the release date
dataset_profitable['date_x'] = pd.to_datetime(dataset_profitable['date_x'])
dataset_profitable['year'] = dataset_profitable['date_x'].dt.year

# Convert categorical variables to dummy variables
dataset_profitable = pd.get_dummies(dataset_profitable, columns=['genre', 'country', 'orig_lang'], drop_first=True)

# Define the features and target variable
X = dataset_profitable.drop(columns=['names', 'date_x', 'overview', 'crew', 'orig_title', 'status', 'revenue'])
y = dataset_profitable['revenue'] > dataset_profitable['revenue'].mean()

# Split the dataset into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Logistic Regression Metrics:")
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))



# Vaihe 5
### Analyysin tulkinta

# Vaihe 6
### Tulosten hyödyntäminen